# **AutoML com AutoKeras**
- O Automated Machine Learning é também conhecido como AutoML;
- Ferramentas e serviços que abstraem os detalhes e conhecimentos necessários para se realizar ML/DL;
- Automatiza-se determinadas tarefas necessárias no desenvolvimento de algum modelo;
- Geralmente, cobrem etapas de normalização de dados e engenharia de “features”; treinamento de modelos de diferentes tipos e com diferentes hiperparâmetros; avaliação e comparação de resultados;
- Algumas ferramentas: H2O, AutoKeras e TPOT;

- Vantagens:  
  - Necessário conhecimento superficial de ML/DL para se utilizar as ferramentas;
  - Diminui o tempo para se ter resultados iniciais;
  - Pode-se utilizar em diferentes tipos de dados e problemas (baseline);
  - Algumas ferramentas possibilitam customizar modelos/layers;
- Desvantagens:
  - Customização que pode ser limitada ou dificultada;
  - Pode-se não obter resultado bom o bastante para o problema, precisando da intervenção de especialistas;



##**Instalação da biblioteca AutoKeras**

In [ ]:
%%capture
!pip install autokeras

##**Import dos pacotes necessários**

In [ ]:
import os
import sklearn
import tensorflow
import pandas as pd
import autokeras as ak

from tensorflow import keras
from keras import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

##**Cópia do dataset**

In [ ]:
%%time
!rm -r lectures-cdas-2023
!git clone https://github.com/renansantosmendes/lectures-cdas-2023.git

##**Leitura do dataset em um DataFrame do Pandas**

In [ ]:
data = pd.read_csv(os.path.join('lectures-cdas-2023', 'fetal_health.csv'))

##**Processamento dos dados**
Nesta célula serão feitos os seguintes passos:
- Separação das variáveis de entrada e saída
- Padronização dos valores das variáveis de entrada
- Separação em dados de treino e teste
- Categorização dos dados de saída (a serem preditos)

In [ ]:
X=data.drop(["fetal_health"], axis=1)
y=data["fetal_health"]

columns_names = list(X.columns)
X_df = StandardScaler().fit_transform(X)
X_df = pd.DataFrame(X_df, columns=columns_names)

X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.3, random_state=42)
y_train=keras.utils.to_categorical(y_train)[:,1:]
y_test=keras.utils.to_categorical(y_test)[:,1:]

##**Criação do modelo (atualizado)**
A partir da versão 2.0.0 do AutoKeras a classe StructuredDataClassifier foi descontinuada e uma forma de contornar é por meio do uso da classe AutoModel em conjunto com a ClassificationHead. Dessa forma o modelo poderá ser treinado conforme o código abaixo

In [ ]:
%%time
input_node = ak.Input()
output_node = ak.ClassificationHead(num_classes=3)(input_node)
auto_model = ak.AutoModel(inputs=input_node,
                          outputs=output_node,
                          max_trials=10,
                          overwrite=True)

auto_model.fit(X_train.values, y_train, epochs=10)

##**Exportando o melhor modelo encontrado até o momento**

In [ ]:
model = auto_model.export_model()

##**Vendo o resumo do melhor modelo**

In [ ]:
model.summary()

##**Avaliando o modelo encontrado com os dados de teste**

In [ ]:
model.evaluate(X_test, y_test)